In [4]:
import Pkg; Pkg.add("FFTW")

using FFTW, Plots

   Resolving package versions...
  No Changes to `~/.julia/environments/v1.7/Project.toml`
  No Changes to `~/.julia/environments/v1.7/Manifest.toml`


[2.3118693751789045e-15, -2.4508404857715234e-15, -9.521461806981277e-15, 3.008559544062215e-15, 4.429449721048407e-15, 2.0497793821768183e-15, -4.934301695495705e-16, -3.4128288140831824e-16]
[-0.0, -1.0000000000000038, -0.9999999999999968, 5.789859109223663e-15, 5.273559366969494e-16, -2.536813575465011e-15, 8.881784197001252e-16, 1.0000000000000002]


In [14]:
N = 63;
P = 1.0;
Δt = P / N
x = 0.0:Δt:(P-Δt)   # lenght(x) == N
y = [sin(2π*7*t) + sin(2π*15*t) + cos(2π*30*t) for t in x] # mixture of simple wave signal
plot(x, y, legend = false, linewidth=2)
Fy = fft(y)[1:N÷2]
ak =  2/N * real.(Fy)
bk = -2/N * imag.(Fy)  # fft sign convention
ak[1] = ak[1]/2
yr = zeros(N,1)
for i in 1:N÷2
    yr .+= ak[i] * cos.(2π*(i-1)/P * x) .+ bk[i] * sin.(2π*(i-1)/P * x)
end
plot!(x, yr, linestyle=:dash, linewidth=2)
ak[ak.<1e-14] .= 0.0
bk[bk.<1e-14] .= 0.0
println(ak)
println(bk)

[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0000000000000027]
[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.000000000000001, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.9999999999999992, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.698478961283989e-14]


In [54]:
function fourier_decompose(f, M)
    N = M+1
    Δθ = π / (N+1)
    x = 0.0:Δθ:(2π-Δθ)
    y = [f(θ) for θ in x]
    Fy = fft(y)[1:N]
    Cc =  real.(Fy) / (N + 1)
    Sc = -imag.(Fy) / (N + 1)  # fft sign convention
    Cc[1] = 0.5 * Cc[1]
    Sc[1] = 0.0
    return Cc, Sc
end

fourier_decompose (generic function with 1 method)

In [57]:
N = 10
C = randn(N+1)
S = randn(N+1)
S[1] = 0.0

function f(θ)
    ms = 0:N
    return sum(C.*cos.(ms .* θ) .+ S.*sin.(ms .* θ))
end
Cc, Sc = fourier_decompose(f, N)
println(C-Cc)
println(S-Sc)

[1.9984014443252818e-15, 4.163336342344337e-16, 3.4416913763379853e-15, 6.661338147750939e-16, 1.9984014443252818e-15, 0.0, 0.0, -1.5543122344752192e-15, -3.164135620181696e-15, 1.609823385706477e-15, -5.88418203051333e-15]
[0.0, -3.3306690738754696e-16, -1.1102230246251565e-15, -2.6645352591003757e-15, 0.0, -3.774758283725532e-15, -2.220446049250313e-15, -2.55351295663786e-15, -1.1102230246251565e-16, -2.4424906541753444e-15, 1.5543122344752192e-15]
